Link to dataset: https://www.kaggle.com/datasets/julienjta/twitter-mentions-volumes

# Statistics Based Anomaly Detection
### 1. Standard Deviation Method
### 2. Z-score
### 3. Inter-Quartile Range (IQR)

In [18]:
# Import Libraries

import pandas as pd
from scipy.stats import zscore
import plotly.express as px

import warnings

warnings.filterwarnings("ignore")

# Data Preprocessing

In [2]:
# Read the data from csv file. We will be looking at Tweets for Apple, Amazon and Google
DATASET_PATH = "../../../dataset/timeseries_company_mentions.csv"

raw_df = pd.read_csv(
    DATASET_PATH,
    usecols=["timestamp", "Apple", "Amazon", "Google"],
)

# make a list of the company names for easy usage later on
company_names = ["Apple", "Amazon", "Google"]

In [4]:
# Check for datatype of columns
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15902 entries, 0 to 15901
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  15902 non-null  object 
 1   Apple      15902 non-null  int64  
 2   Amazon     15831 non-null  float64
 3   Google     15842 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 497.1+ KB


In [5]:
# Let us convert datetime to datetime and hour for grouping all the tweet mentions within that particular hour
def datetime_to_datehour(datetime_val: str) -> str:
    return f"{datetime_val.split(':')[0]}:00:00"


# applying the custom function to the timestamp data
raw_df.timestamp = raw_df.timestamp.map(lambda val: datetime_to_datehour(val))

In [7]:
# convert timestamp to datetime
raw_df["timestamp"] = pd.to_datetime(raw_df["timestamp"])

In [8]:
# Check for missing vals
raw_df.isna().sum()

timestamp     0
Apple         0
Amazon       71
Google       60
dtype: int64

In [9]:
# drop missing values
raw_df.dropna(inplace=True)

In [10]:
# recheck missing values
raw_df.isna().sum()

timestamp    0
Apple        0
Amazon       0
Google       0
dtype: int64

In [11]:
# group tweet mentions by hour
raw_df = raw_df.groupby(by="timestamp", as_index=False).sum()

In [12]:
# convert datetime to datetime data type
raw_df.timestamp = pd.to_datetime(raw_df.timestamp)

# 1. Standard Deviation Method

* upper limit = mean + 3 * std dev<br>
* lower limit = mean - 3 * std dev

In [28]:
# Get mean and std deviation values
mean = raw_df.mean()
std_dev = raw_df.std()

In [43]:
# calculate upper and lower limits
upper_limit_std_dev = mean + 2 * std_dev
lower_limit_std_dev = mean - 2 * std_dev

In [37]:
# write function to check if the value is Normal or Anomaly
def check(value: int, upper_limit_value: float, lower_limit_value: float) -> str:
    if lower_limit_value < value < upper_limit_value:
        return "Normal"
    return "Anomaly"

In [46]:
# iterate over the 3 companies in company_names list
for company in company_names:
    # form a dataframe for single company with timestamp and tweet mention count
    company_df = raw_df[["timestamp", company]].copy()
    # create a column which tells whether the values are normal or anomaly
    company_df["anomaly"] = company_df[company].map(
        lambda value: check(
            value, upper_limit_std_dev[company], lower_limit_std_dev[company]
        )
    )

    # graphical representation
    fig = px.scatter(data_frame=company_df, x="timestamp", y=company, color="anomaly")
    fig.update_layout(
        title=f"Anomaly Detection using Standard Deviation for {company}",
        xaxis_title="DateTime",
        yaxis_title="Twitter Mentions",
    )
    fig.show()

# 2. Z-score
* upper cutoff score: z = USUALLY between 2 and 3
* lower cutoff score: z = USUALLY between -2 and -3

In [49]:
# set Zscore threshold
ZSCORE_THRESHOLD = 2.5

# iterate over the 3 companies in company_names list
for company in company_names:
    # form a dataframe for single company with timestamp and tweet mention count
    company_df = raw_df[["timestamp", company]].copy()
    # Calculate Zscore
    company_df["zscore"] = zscore(company_df[company])
    # create a column which tells whether the values are normal or anomaly
    company_df["anomaly"] = company_df["zscore"].map(
        lambda value: "Normal"
        if -ZSCORE_THRESHOLD < value < ZSCORE_THRESHOLD
        else "Anomaly"
    )

    # graphical representation
    fig = px.scatter(data_frame=company_df, x="timestamp", y=company, color="anomaly")
    fig.update_layout(
        title=f"Anomaly Detection using Z-score for {company}",
        xaxis_title="DateTime",
        yaxis_title="Twitter Mentions",
    )
    fig.show()

# 3. Statistics: Inter-Quartile Range (IQR)
* upper limit = Q3 + 1.5 * IQR
* lower limit = Q1 - 1.5 * IQR

In [52]:
# First Quartile
Q1 = raw_df.quantile(0.25)
# Third Quartile
Q3 = raw_df.quantile(0.75)
# Inter-Quartile Range (IQR)
IQR = Q3 - Q1

# Upper and Lower limit based on IQR
upper_limit_iqr = Q3 + 1.5 * IQR
lower_limit_iqr = Q1 - 1.5 * IQR

In [53]:
for company in company_names:
    # form a dataframe for single company with timestamp and tweet mention count
    company_df = raw_df[["timestamp", company]].copy()
    # create a column which tells whether the values are normal or anomaly
    company_df["anomaly"] = company_df[company].map(
        lambda value: check(value, upper_limit_iqr[company], lower_limit_iqr[company])
    )

    # graphical representation
    fig = px.scatter(data_frame=company_df, x="timestamp", y=company, color="anomaly")
    fig.update_layout(
        title=f"Anomaly Detection using Inter-Quartile Range for {company}",
        xaxis_title="DateTime",
        yaxis_title="Twitter Mentions",
    )
    fig.show()